## Project Dublin Bus 


## Change the data format, remove duplication and  save it 

## Import package and data 

In [18]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib  for visualisation/plotting
import matplotlib.pyplot as plt

#import matplotlib.patches as mpatches
import seaborn as sns

#import package matplotlib for import plots to PDF
from matplotlib.backends.backend_pdf import PdfPages

#For showing plots directly in the notebook
#%matplotlib inline

# Reading from a csv file, into a data frame

import time 
begin_time=time.time()

In [19]:
#Change the columns's names 
Columns_name = ['Timestamp','Line_ID','Direction','Journey_Pattern_ID','Date','Vehicle_Journey_ID','Operator',
              'Congestion','Lon_WGS84','Lat_WGS84','Delay','Block_ID','Vehicle_ID','Stop_ID','At_Stop']

In [20]:
#This is the data of January 2013
df = pd.read_csv('dublin_2012_week1_before.csv',names=Columns_name, dtype={ 'Vehicle_Journey_ID': np.int64} )
df.head(3)

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160000000000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0
1,1352160000000000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0
2,1352160000000000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0


In [21]:
# Check how many rows we have after loaded the csv file 
df.shape

(10856935, 15)

#  Data Cleaning 


In [5]:
#Drop duplicate rows 
df=df.drop_duplicates()
#Drop constant column
#df.drop(['Direction','Operator'],axis=1,inplace=True)
#Drop Unused column
#df.drop(['Line_ID','Congestion','Block_ID','Vehicle_ID'],axis=1,inplace=True)

In [6]:
# Check how many rows we have after remove duplication
df.shape

(10856935, 15)

## Remove all Journey_Pattern_ID is null

In [7]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'].isnull()])

In [8]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(10856077, 15)

In [9]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'] == 'null'])

In [10]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(9211192, 15)

In [8]:
#Print all the unique Journey_Pattern_ID
#print("Unique Journey_Pattern_ID ", pd.unique(df.Journey_Pattern_ID.ravel()))

## Remove some noise data ( Like only have less than 45 rows in one trip  )
## Those data are not actually a full run 

In [11]:
df_groupby=df.groupby(['Vehicle_Journey_ID','Date','Journey_Pattern_ID'])

#size = df_groupby.size()
#size=size[size<13]

#keys=df_groupby.groups

# Filter all the data only have less than 13 rows data 
df_short_Journ=df_groupby.filter(lambda x: len(x) < 45 )

In [12]:
# Drop all the Journery that are less than a number 

df=pd.concat([df, df_short_Journ]).drop_duplicates(keep=False)

In [13]:
#Check how many rows we deleted
df.shape


(9160370, 15)

# Solution to check the long time trip issue
## Check the time gap of two gps coordinations, if there are big than one minute, delete the first one 

In [9]:
df['Timestamp'] = df['Timestamp'] //1000000
df.head(3)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0


In [10]:
df['Next_Timestamp']=df.groupby(['Vehicle_Journey_ID','Journey_Pattern_ID','Date'])['Timestamp'].shift(-1)

In [11]:
def get_rid_of_nulls(row):
    if pd.isnull(row['Next_Timestamp']):
        return row['Timestamp']
    else:
        return row['Next_Timestamp']

df['Next_Timestamp']=df.apply(get_rid_of_nulls, axis=1)

CPU times: user 6min 13s, sys: 49.4 s, total: 7min 3s
Wall time: 8min 28s


In [12]:
df.tail()

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Next_Timestamp
10856930,1352764796,102,0,01020001,2012-11-12,415,HN,0,-6.250725,53.439415,-564,102005,33276,815,0,1.352765e+09
10856931,1352764796,33,0,00331005,2012-11-12,269,SL,0,-6.109184,53.580326,-36,33011,43052,3818,0,1.352765e+09
10856932,1352764796,38,0,00380001,2012-11-12,1269,PO,0,-6.375340,53.399990,154,38010,33596,1819,0,1.352765e+09
10856933,1352764798,15,0,015A0001,2012-11-12,3130,RD,0,-6.299134,53.311970,267,15009,24623,1089,0,1.352765e+09
10856934,1352764798,7,0,00070001,2012-11-12,6963,D1,0,-6.138767,53.276184,-128,7014,43007,2040,1,1.352765e+09


In [13]:
df['Next_Timestamp'] = df.Next_Timestamp.astype(int)

In [14]:
df['Between_Time']=df['Next_Timestamp']-df['Timestamp']

CPU times: user 61.5 ms, sys: 122 ms, total: 184 ms
Wall time: 287 ms


In [18]:
df_long=df.loc[(df['Between_Time']>120)]

In [24]:
df_long.head(15)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Next_Timestamp,Between_Time
2576,1352160349,56,0,056A1001,2012-11-05,2063,RD,0,-6.237294,53.342300,-616,56001,33230,353,0,1352161087,738
4562,1352160728,39,0,039A0001,2012-11-05,3705,PO,0,-6.273350,53.352333,-731,39008,33595,7011,0,1352161108,380
6057,1352161108,39,0,039A0001,2012-11-05,3705,PO,0,-6.274783,53.353317,-731,39008,33595,7011,0,1352161287,179
6872,1352161386,39,0,039A0001,2012-11-05,3705,PO,0,-6.274783,53.353333,-731,39008,33595,7011,0,1352161528,142
7439,1352161627,39,0,039A0001,2012-11-05,3705,PO,0,-6.274766,53.353317,-731,39008,33595,7011,0,1352161748,121
8801,1352162737,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352163167,430
8933,1352163087,39,0,039A0001,2012-11-05,3705,PO,0,-6.274800,53.353333,-731,39008,33595,7011,0,1352163368,281
8956,1352163167,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352163586,419
8984,1352163302,79,0,00790001,2012-11-05,3511,CD,0,-6.296617,53.347698,29,79003,33191,2907,0,1352163730,428
9036,1352163586,13,0,00130008,2012-11-05,4094,HN,0,-6.276967,53.416916,-531,13103,33338,2166,0,1352169353,5767


In [20]:
df_long.shape

(6760, 17)

In [23]:
df=df.drop(df.index[df['Between_Time']>120])

In [17]:
# Drop the feature we don't need now 
df.drop(['Next_Timestamp','Between_Time'],axis=1,inplace=True)

In [24]:
#Check how many rows we deleted(Before it is (9211192, 15) )
df.shape

(9204432, 17)

# Add Distance feature 

In [24]:
df=df.loc[df['Journey_Pattern_ID']=='041B0002']
df.reset_index(drop=True, inplace=True)

In [25]:
df

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160002000000,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1
1,1352160023000000,41,0,041B0002,2012-11-05,61,SL,0,-6.261317,53.451035,-623,41008,33631,3874,1
2,1352160041000000,41,0,041B0002,2012-11-05,61,SL,0,-6.261350,53.448318,-623,41008,33631,3874,1
3,1352160061000000,41,0,041B0002,2012-11-05,61,SL,0,-6.263017,53.445648,-623,41008,33631,3874,1
4,1352160083000000,41,0,041B0002,2012-11-05,61,SL,0,-6.262866,53.443317,-623,41008,33631,3874,1
5,1352160102000000,41,0,041B0002,2012-11-05,61,SL,0,-6.263933,53.440716,-623,41008,33631,3874,1
6,1352160122000000,41,0,041B0002,2012-11-05,61,SL,0,-6.260933,53.440048,-623,41008,33631,3874,1
7,1352160142000000,41,0,041B0002,2012-11-05,61,SL,0,-6.255700,53.439434,-623,41008,33631,3874,1
8,1352160164000000,41,0,041B0002,2012-11-05,61,SL,0,-6.252000,53.437050,-623,41008,33631,3874,1
9,1352160183000000,41,0,041B0002,2012-11-05,61,SL,0,-6.246867,53.436481,-623,41008,33631,3874,1


In [9]:
df.shape

(5246, 15)

In [28]:
from geopy.distance import distance

def coor():
    prev = yield
    running_dist = 0
    while True:
        next_ = yield running_dist
        running_dist += distance(prev, next_).meters
        prev = next_
        
def group_df(groupby_params, df):
    """
    :param groupby_params: list of feature names as strings by which to group by
    :param df: dataframe that is to be grouped
    :return: a pandas groupby object
    """
    if len(groupby_params) == 0:
        print("You did not provide any groupby parameters")
        return None
    else:
        df_grouped = df.groupby(groupby_params)
        return df_grouped
        
def add_distance_all_runs(df):
    """
    :param df: dataframe before distance has been added
    :return: dataframe with distances for each run
    """
    df['Distance'] = -1
    vj_groups = []
    df_grouped = group_df(['Journey_Pattern_ID','Date'], df)
    for journey_name, journey_group in df_grouped:
        vehicle_groups = group_df(['Vehicle_Journey_ID'], journey_group)
        for v_name, v_group in vehicle_groups:
            c = coor()
            next(c)
            for index, row in v_group.iterrows():
                dist = c.send((row['Lat_WGS84'], row['Lon_WGS84']))
                v_group.set_value(index, 'Distance', dist)
            if v_group['Distance'].max() < 36000:
                vj_groups.append(v_group)

    return pd.concat(vj_groups)

%time df=add_distance_all_runs(df)

CPU times: user 1.04 s, sys: 13.2 ms, total: 1.05 s
Wall time: 1.07 s


In [32]:
df.to_csv('a_trip.csv')

In [26]:
#Convert Timestamp to datetime and new column datetime to data frame 
df['datetime'] = pd.to_datetime(df['Timestamp']//1000000,unit='s')
df

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime
0,1352160002000000,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,2012-11-06 00:00:02
1,1352160023000000,41,0,041B0002,2012-11-05,61,SL,0,-6.261317,53.451035,-623,41008,33631,3874,1,2012-11-06 00:00:23
2,1352160041000000,41,0,041B0002,2012-11-05,61,SL,0,-6.261350,53.448318,-623,41008,33631,3874,1,2012-11-06 00:00:41
3,1352160061000000,41,0,041B0002,2012-11-05,61,SL,0,-6.263017,53.445648,-623,41008,33631,3874,1,2012-11-06 00:01:01
4,1352160083000000,41,0,041B0002,2012-11-05,61,SL,0,-6.262866,53.443317,-623,41008,33631,3874,1,2012-11-06 00:01:23
5,1352160102000000,41,0,041B0002,2012-11-05,61,SL,0,-6.263933,53.440716,-623,41008,33631,3874,1,2012-11-06 00:01:42
6,1352160122000000,41,0,041B0002,2012-11-05,61,SL,0,-6.260933,53.440048,-623,41008,33631,3874,1,2012-11-06 00:02:02
7,1352160142000000,41,0,041B0002,2012-11-05,61,SL,0,-6.255700,53.439434,-623,41008,33631,3874,1,2012-11-06 00:02:22
8,1352160164000000,41,0,041B0002,2012-11-05,61,SL,0,-6.252000,53.437050,-623,41008,33631,3874,1,2012-11-06 00:02:44
9,1352160183000000,41,0,041B0002,2012-11-05,61,SL,0,-6.246867,53.436481,-623,41008,33631,3874,1,2012-11-06 00:03:03


# For some run, the trip is too long, then we could remove all the row after three and half hours. As some bus didn't stop after finish one trip 

In [ ]:
# Add new feature Trip_Time 
zscore=lambda x: (x - x.min())
df['Trip_Time']= df.groupby(['Vehicle_Journey_ID','Journey_Pattern_ID','Date'])['Timestamp'].transform(zscore)

In [ ]:
# Drop all the rows' Trip_Time is longer than four hours 
df=df.drop(df.index[df['Trip_Time']>=14400])

# Add some other features 

In [ ]:
#Convert Timestamp to datetime and new column datetime to data frame 
df['datetime'] = pd.to_datetime(df['Timestamp'],unit='s')
df.head(5)

In [ ]:
#ADD day of the week to the date frame 
df['day_of_week'] = df['datetime'].dt.weekday_name
df.head(3)

In [ ]:
# Only keep the first row for every trip at the same Stop(keep both at or not_at stop)
df = df.drop_duplicates(['Vehicle_Journey_ID','Journey_Pattern_ID','Date','Stop_ID','At_Stop'])

# Save the file for modeling 

In [27]:
#Save the data for next step
df.to_csv('dublin_2012_week1.csv', header=False,index=False)

In [ ]:
End_time=time.time()
Duringtime=End_time-begin_time
print("The time used is ",Duringtime//60)